In [3]:
base_folder = "/mnt/e/SteamLibrary2/steamapps/common/SlayTheSpire/SpeedTheSpire/DEBUG"

In [22]:
import tensorflow as tf

# modified from code from https://machinelearningmastery.com/lstm-autoencoders/

# lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

In [5]:
# https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join
datafiles = [f for f in listdir(base_folder) if isfile(join(base_folder, f))]
len(datafiles)

103

In [6]:
corpus = []

for datafilename in datafiles:
    fullpath = join(base_folder, datafilename)
    with open(fullpath, "r") as datafile:
        lines = datafile.readlines()
        for line in lines:
            if line[0] == "I":
                corpus.append(line[1:-1])
len(corpus)

8919

In [7]:
corpus = [[char for char in line] for line in corpus]

In [8]:
import string

alphabet = " ?%!.$\\:_-(){}[]\",+" + string.ascii_lowercase + string.ascii_uppercase + string.digits
char_to_idx = {c: alphabet.find(c) for c in alphabet}
print(char_to_idx)

{' ': 0, '?': 1, '%': 2, '!': 3, '.': 4, '$': 5, '\\': 6, ':': 7, '_': 8, '-': 9, '(': 10, ')': 11, '{': 12, '}': 13, '[': 14, ']': 15, '"': 16, ',': 17, '+': 18, 'a': 19, 'b': 20, 'c': 21, 'd': 22, 'e': 23, 'f': 24, 'g': 25, 'h': 26, 'i': 27, 'j': 28, 'k': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'w': 41, 'x': 42, 'y': 43, 'z': 44, 'A': 45, 'B': 46, 'C': 47, 'D': 48, 'E': 49, 'F': 50, 'G': 51, 'H': 52, 'I': 53, 'J': 54, 'K': 55, 'L': 56, 'M': 57, 'N': 58, 'O': 59, 'P': 60, 'Q': 61, 'R': 62, 'S': 63, 'T': 64, 'U': 65, 'V': 66, 'W': 67, 'X': 68, 'Y': 69, 'Z': 70, '0': 71, '1': 72, '2': 73, '3': 74, '4': 75, '5': 76, '6': 77, '7': 78, '8': 79, '9': 80}


In [9]:
for line in corpus:
    for char in line:
        if char not in alphabet:
            assert char in alphabet

In [14]:
def one_hot_encode(line):
    result = []
    for char in line:
        row = [0] * len(alphabet)
        row[char_to_idx[char]] = 1
        result.append(row)
    return result

def idx_encode(line):
    result = [char_to_idx[char] for char in line]
    return result

def sparse_tensor_encode(line):
    assert False, "TODO: implement sparse_tensor_encode"
    result = tf.sparse.SparseTensor(indices=[],
                                    values = [],
                                    dense_shape=[len(line), len(alphabet)])
    return result

In [27]:
# define input sequence
seq_in = [idx_encode(line) for line in corpus]

In [ ]:
# adapted from https://blog.keras.io/building-autoencoders-in-keras.html

timesteps = None  # Length of your sequences
input_dim = 1
latent_dim = 30

inputs = keras.Input(shape=(timesteps, input_dim))
encoded = layers.LSTM(latent_dim)(inputs)

decoded = layers.RepeatVector(timesteps)(encoded)
decoded = layers.LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = keras.Model(inputs, decoded)
encoder = keras.Model(inputs, encoded)


In [34]:
# reproduction of inputs attempt
num_samples = len(seq_in)

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(len(seq_in[0]),1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(array(seq_in[0]), array(seq_in[0]), epochs=300, verbose=2)
plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

Epoch 1/300


ValueError: in user code:

    File "/home/sinan/Projects.git/decode-the-spire/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/sinan/Projects.git/decode-the-spire/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/sinan/Projects.git/decode-the-spire/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/sinan/Projects.git/decode-the-spire/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/sinan/Projects.git/decode-the-spire/venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/ho